# 情報科学演習II アルゴリズムとデータ構造II 第2週 1冊目

#### 第2週目次

1. ダイクストラのアルゴリズムの簡単な実装（1冊目）
2. 実装のクラス化（2冊目）
3. 優先度付きキュー（3冊目）
4. ダイクストラのアルゴリズムの効率的な実装（4冊目）

## 1. ダイクストラのアルゴリズムの簡単な実装

ダイクストラのアルゴリズムは単一始点の最短経路問題を効率よく解く解法です．
この章ではリストを使った簡単な実装をします．
第1週4冊目の6.3節で示した疑似コードをPythonで書きます．
アルゴリズムの詳細は第1週6.3節で示しています．
第1週6.3節を横に表示しながらこの章を進めるのがよいでしょう．

優先度付きキューを用いた効率的な実装は4章でします．

### 1.1 準備とnetworkxの復習

次の二つのセルは準備ですので，実行してください．
二つ目のセルで必要なモジュールを`import`し，いくつかの関数を定義しておきます．

In [ ]:
!apt install libgraphviz-dev
!pip install pygraphviz
# このセルの実行は1分近くかかるし出力も多い（100行近い）
# 最終行は Successfully installed pygraphviz-x.xx
# x.xxの部分は1.13などのバージョン番号
# 2度目の実行は短時間で終わり，出力は10行程度である．2度実行した場合の最終行は次のようになる．(y.yyとx.xx)はバージョン番号
# Requirement already satisfied: pygraphviz in /usr/local/lib/pythony.yy/dist-packages (x.xx)

In [ ]:
import networkx as nx
import math
import pygraphviz as pgv
from IPython.display import Image, display_png # Imageクラス
from test import support # sortdict関数

# グラフの頂点のリストと辺のリストを出力する関数
def print_graph(G): # GはnetworkxのGraphクラス (networkx.classes.graph.Graphクラス)
  print('print_graph:')
  print('  nodes', list(G.nodes))
  print('  edges', list(G.edges)) # 辺の重みも出力するにはedgesの代わりにedges.data()
  
# グラフを図示する関数
def draw_graph(G):
  A = nx.nx_agraph.to_agraph(G)
  for edge in G.edges(data='weight'):
    u, v, d = edge
    A.get_edge(u, v).attr['label'] = d

  A.layout(args='-Nshape=circle')  # default prog='neato' 
  A.draw('tmp.png')
  display_png(Image('tmp.png'))

# グラフのサンプル
def sample_graph1():
  G = nx.Graph() # 無向グラフ
  for i,j,w in [[0, 1, 5], [0, 4, 9], [0, 7, 8], [1, 2, 12], [1, 3, 15], 
                [1, 7, 4], [2, 3, 3], [2, 6, 11], [3, 6, 9], [4, 5, 4], 
                [4, 7, 5], [5, 2, 1], [5, 6, 13], [7, 2, 7], [7, 5, 6]]:
    G.add_edge(i, j, weight=w) # weightはnetworkx, labelはgraphviz
  return G

G1 = sample_graph1()
L1 = nx.get_edge_attributes(G1, 'weight') # L1は辞書型でキーは辺，値は重み（辺の長さ），
answer_d1 = nx.single_source_dijkstra_path_length(G1, 0)
print('G1の辺の長さ L1 = ', L1)
print('グラフ G1')
draw_graph(G1)


networkxの復習をします．
上に図示した7頂点のグラフG1を使います．

**練習**
次のコードセルでグラフG1の頂点のリストを出力しなさい．
`list(G1.nodes)`を使います．
`[0, 1, 4, 7, 2, 3, 6, 5]`が出力されたら正解です．

In [ ]:
# 頂点のリスト


**練習**
次のコードセルでグラフG1の頂点0の隣接頂点のリストを出力しなさい．
`list(G1.adj[0])`を使います．
`[1, 4, 7]`が出力されたら正解です．

頂点7の隣接頂点は`[0, 1, 2, 4, 5]`です．
順番は問いません．

In [ ]:
# 隣接頂点のリスト


`relax`関数で辺の長さを読み出しますので，辺の長さを読み出す練習をします．
辺の長さが格納されているデータ型は辞書です．

**練習**
グラフG1には辺(0, 1)があり，この辺の長さは5です．
グラフG1の辺の長さ（重み）は辞書`L1`に格納されています．
次の3つのセルを実行するとわかりますが，`L1`のキーとして`(0, 1)`はあるが`(1, 0)`はありません．
`i`と`j`を変更して他の辺(2, 7)についても一方のキーしかないことを確かめなさい．

In [ ]:
# L1のキーのリスト
list(L1)

In [ ]:
i = 0
j = 1
L1[(i, j)] # 辺(i, j)の長さ

In [ ]:
L1[(j, i)] # 辺(j, i)の長さ

In [ ]:
(i, j) in L1 # 辺(i, j)が辞書L1に含まれているか

In [ ]:
(j, i) in L1 # 辺(j, i)が辞書L1に含まれているか

**練習**
グラフG1の辺(u, v)の長さを辞書`L1`で調べて変数luvに格納するコードになるように次のセルの`....`の部分を書きなさい．
`L1`のキーに(u, v)がある場合と(v, u)がある場合があるので`if`文で場合分けすること．

u = 0, v = 1だと辺(u, v)の長さ5を出力すること．
u = 1, v = 0に書き換えても，辺(u, v)の長さ5を出力すること．
u = 5, v = 6 や u = 6, v = 5に書き換えると辺(u, v)の長さ13を出力すること．

In [ ]:
L = L1
u = 0
v = 1
if .... :
  luv = ....
else:
  luv = ....
luv

### 1.2 dijkstra関数をPythonで書く

第1週6.3節の疑似コードをPythonのコードに書き換えます．
次のコードセルに前回6.3節のdijkstra関数をコピーしてあります．
試しに実行しても文法エラー(SyntaxError)で関数が定義できませんね．

**練習**
次のコードのdijkstra関数を上から順に読んで，Python言語に合うように書き換えなさい．
次のように書き換えればよいでしょう．

- 関数を定義するときは関数名の前に`def`をつけます．
  
  `def dijkstra(G, L, s):`
- `頂点u`という日本語混じりの変数名を`u`に書き換えます．
- 頂点集合`V`をnetworkxの頂点のリストに書き換えます．
  グラフ`G`の頂点のリストは，1.1節で練習したように`list(G.nodes)`です．
- `Sにuを加える`は`S`が集合型ですから`S.add(u)`と書き換えます．
- `頂点v`という日本語混じりの変数名を`v`に書き換えます．
- `uの隣接頂点`をnetworkxの隣接頂点リストに書き換えます．
  1.1節で練習したように`list(G.adj[u])`がグラフ`G`の頂点`u`の隣接頂点リストです．

In [ ]:
d = {} # d[v]は頂点sから頂点vまでの距離
Q = [] # 距離が確定していない頂点のリスト
pi = {} # pi[v]は短い経路上で，vの直前にある頂点

# sから各頂点への最短距離を返す
# G: グラフ, L: Gの各辺の長さの辞書, s: 始点とする頂点
dijkstra(G, L, s):
  initialize_single_source(G) # すべての頂点uについて距離d[u]を初期化する
  S = set() # Sは空集合
  
  # キューQの初期化
  for 頂点u in V: # Vに含まれるそれぞれの頂点uに対して
    insert(u) # Qに頂点uを加える

  # 始点sの距離を0にする
  d[s] = 0

  # 最短距離を求める
  while Q が空集合でない: # Qが空集合になるまで
    頂点u = extract_min()   #   Qから最小値の頂点uを取り出す
    S に u を加える       #   Sに頂点uを加える
    for 頂点v in uの隣接頂点: #   uに隣接するそれぞれの頂点に対して
      relax(u, v, L) #     頂点vへの二つの経路のうち短いものを残しキューの並びも調整する

上のセルの`dijkstra`関数をPython言語に書き換えると，このセルを実行しても文法エラー(SyntaxError)は起きません．

次のコードセルで作成した`dijkstra`関数を呼び出してみましょう．
`initialize_single_source`など未定義の関数があるので呼び出すとエラー(NameError)になります．

In [ ]:
dijkstra(G1, L1, 0)

`dijkstra`関数から呼び出す関数をこれから作成します．
このノートブックの終わりでは`dijkstra`関数が動作するようにしましょう．
前回6.3節の疑似コードを確認しながら次の4つの関数を書いてください．

- initialize_single_source(G)
- insert(u)
- extract_min()
- relax(u, v, L)

**練習**
`initialize_single_source(G)`関数を次のセルに定義しなさい．
引数の`G`はnetworkxで表した無向グラフです．
無限大は`math.inf`と書きます．

In [ ]:
# すべての頂点uについて距離d[u]を初期化する．距離は無限大にする．
def initialize_single_source(G):
  for 頂点v in V: # VはグラフGの頂点の集合
    d[v] = 無限大
    pi[v] = -1

次のセルで`initialize_single_source`関数の動作を確認しましょう．
出力が次の2行のように慣れば期待したように動作しています．
頂点の並び順は問いません．

```
d =  {0: inf, 1: inf, 2: inf, 3: inf, 4: inf, 5: inf, 6: inf, 7: inf}
pi =  {0: -1, 1: -1, 2: -1, 3: -1, 4: -1, 5: -1, 6: -1, 7: -1}
```

In [ ]:
d = {}
pi = {}
initialize_single_source(G1)
print('d = ', d)
print('pi = ', pi)

**練習**
`insert(u)`関数を次のセルに定義しなさい．
`u`は頂点（の番号）です．

ヒント：`Q`は頂点のリストです．
リストに要素を追加するには`append`関数を使います．
詳しくは第1週の4.1節や[リストの説明2](https://docs.python.org/ja/3/tutorial/datastructures.html#more-on-lists)を読んでください．

In [ ]:
# Qに頂点uを加える
def insert(u):
  キューQに頂点uを加える．

次のセルを実行して`insert`関数の動作を確認しましょう．
リスト`Q`の長さが2から3に一つ増えることだけ確認することにします．
`2 -> 3`と出力されたらできたことにしましょう．

いい加減な確認しかしていないので，あとで`dijkstra`関数の動作が期待と異なるときはこの関数を修正する必要があるかもしれません．

In [ ]:
Q = [1, 5]
print(len(Q), end = ' -> ')
insert(3)
print(len(Q))

**練習**
リスト`Q`のすべての頂点の距離が出力されるように，次のコードの`....`の部分を書き換えなさい．
`for`文を使って`extract_min()`関数でこの`for`文を使います．

In [ ]:
for v in ....:
  print(d[v])

**練習**
`extract_min()`関数を次のセルに定義しなさい．
この関数は最短距離の頂点を返します．
最短距離の頂点はリスト`Q`から削除します．

ヒント：
`Q`は距離が確定していない頂点のリストです．
最短距離の頂点を探すには`Q`のすべての頂点`v`の距離`d[v]`を比べる必要があります．
リストの各要素`v`に対して処理を繰り返すには`for v in Q:`のように`for`文を使います．
リスト`Q`から要素を取り除くには`remove`関数を使います．

In [ ]:
# Qから最小値の頂点uを取り出す
def extract_min():
  Qから距離d[v]が最も短い頂点vを探して返す．

次の一連のセルを実行して`extract_min`関数の動作を確認しましょう．
出力の`(i, j)`は頂点iの距離がjであるというふうに読んでください．
コードセルの末尾に期待される出力をコメントで書いていますので，出力が一致することを確認しましょう．

In [ ]:
d = {1: 5, 2: 4, 3: 9, 4: 1, 5: 7}
Q = [1, 5, 3]
u = extract_min()
u, d[u]
# (1, 5)

In [ ]:
Q = [5, 3, 1]
u = extract_min()
u, d[u]
# (1, 5)

In [ ]:
Q = [3, 5, 1]
u = extract_min()
u, d[u]
# (1, 5)

In [ ]:
Q = [2, 3, 5, 1]
u = extract_min()
u, d[u]
# (2, 4)

In [ ]:
Q = [3]
u = extract_min()
u, d[u]
# (3, 9)

In [ ]:
Q = [1, 2, 3, 4, 5]
u = extract_min()
u, d[u]
# (4, 1)

最後の関数は`relax`です．

**練習**
`relax(u, v, L)`関数を次のセルに定義しなさい．
`relax`関数が呼び出されるのは頂点`u`の最短経路が求まったときです．
頂点`v`は頂点`u`の隣の頂点で，頂点`v`にもっと短い経路が見つかったらそれに更新するのがこの関数の役割です．
`L`は辺の長さを格納した辞書です．

頂点`u`と頂点`v`の辺の長さを`luv`とすると，以下の式が成り立つならば，
`u`を通って頂点`v`に至る経路が，これまで見つかった経路より短い`v`への経路です．

$$ d[v] > d[u] + luv $$

より短い経路であれば次のように距離`d[v]`と直前の頂点`pi[v]`を新しいものに更新します．

```
d[v] = d[u] + luv
pi[v] = u
```

In [ ]:
# 頂点vへの二つの経路のうち短いものを残す
def relax(u, v, L):
   # luvに辺(u, v)の長さを代入する
  if (u, v) in L:
    luv = L[(u, v)]
  else:
    luv = L[(v, u)]

  頂点uの距離d[u]に辺(u,v)の長さluvを加えた距離をdv2とする．
  もし，dv2がこれまでの距離d[v]より短ければ，dv2をd[v]に代入し，pi[v]にuを代入する．

### 1.3 dijkstra関数の動作を確かめる

`dijkstra`関数と他4つの関数ができたので実行してみます．

In [ ]:
d = {} # d[v]は頂点sから頂点vまでの距離
Q = [] # 距離が確定していない頂点のリスト
pi = {} # pi[v]は短い経路上で，vの直前にある頂点

s = 0
dijkstra(G1, L1, s)
d

次のセルで`dijkstra`関数で求めた距離が正しいか確認しましょう．
グラフG1のすべての頂点について始点0からの距離を求めています．

In [ ]:
if d == answer_d1:
  print('すべての頂点の距離が一致しました')
else:
  print('距離が間違っている場合があります．')
  print()
  for u, du in d.items():
    if du != answer_d1[u]:
      print(F'始点{s}から頂点{u}への最短距離は{answer_d1[u]}ですが，{du}と求めています．')
  draw_graph(G1)

**デバッグ**
正しく計算できていない場合は，`dijkstra(G, L, s)`関数の`while`ループのところで`Q`と`d`の値を出力して動作を確認するのがいいでしょう．
```
  while len(Q) > 0: # Qが空集合になるまで
    print('Q = ', Q)
    print('d = ', d)
    u = extract_min()   #   Qから最小値の頂点uを取り出す
```